# B-Splines and Trigonmometric Functions

## Sine

The alternating sum of the integer shifts of the first derivative of a
B-spline approaches the sine function as the degree tends to infinity.
More precisely, it holds that

$$\sin(\nu\,\pi)=\lim_{n\rightarrow\infty}\left(-\frac{1}{4}\,\left(\frac{\pi}{2}\right)^{n}
\,\sum_{k\in{\mathbb{Z}}}\,\left(-1\right)^{k}\,\dot{\beta}^{n}(\nu-k)\right).$$
Verify this property by plotting, over the single period $\nu\in[-1,1]$, the difference between the sine function and the spline approximation.

In [67]:
# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_sin_plot (
    degree = 3
):
    # Layout of the plot
    pts = 200 + 1
    (fig, ax) = plt.subplots()
    plt.xlim(-1.025, 1.025)
    plt.ylim(-0.05, 0.05)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)

    # Trigonometric data
    abscissa = np.array(
        [-1 + 2 * k / (pts - 1) for k in range(pts)],
        dtype = float
    )
    sin_data = np.array(
        [math.sin(nu * math.pi) for nu in abscissa],
        dtype = float
    )

    # B-spline approximation of a sine
    normalization = -((math.pi / 2) ** degree) / 4
    spline_data = np.array(
        [
            normalization * math.fsum(
                ((-1) ** k) * sk.grad_b_spline(nu - k, degree)
                for k in range(
                    math.ceil(nu - (degree + 1) / 2),
                    math.floor(nu + (degree + 1) / 2) + 1
                )
            )
            for nu in abscissa
        ],
        dtype = float
    )

    # Plot
    plt.plot(abscissa, sin_data - spline_data)
    fig.tight_layout()
    plt.show()

# Interact with the degree
interactive(b_spline_as_sin_plot, degree = (1, 10))

interactive(children=(IntSlider(value=3, description='degree', max=10, min=1), Output()), _dom_classes=('widge…

## Cosine

The alternating sum of the half-integer shifts of the first derivative of a
B-spline approaches the cosine function as the degree tends to infinity.
More precisely, it holds that

$$\cos(\nu\,\pi)=\lim_{n\rightarrow\infty}\left(-\frac{1}{4}\,\left(\frac{\pi}{2}\right)^{n}
\,\sum_{k\in{\mathbb{Z}}}\,\left(-1\right)^{k}\,\dot{\beta}^{n}(\nu+\frac{1}{2}-k)\right).$$
Verify this property by plotting, over the single period $\nu\in[-1,1]$, the difference between the cosine function and the spline approximation.

In [68]:
# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_cos_plot (
    degree = 3
):
    # Layout of the plot
    pts = 200 + 1
    (fig, ax) = plt.subplots()
    plt.xlim(-1.025, 1.025)
    plt.ylim(-0.05, 0.05)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)

    # Trigonometric data
    abscissa = np.array(
        [-1 + 2 * k / (pts - 1) for k in range(pts)],
        dtype = float
    )
    cos_data = np.array(
        [math.cos(nu * math.pi) for nu in abscissa],
        dtype = float
    )

    # B-spline approximation of a cosine
    normalization = -((math.pi / 2) ** degree) / 4
    spline_data = np.array(
        [
            normalization * math.fsum(
                ((-1) ** k) * sk.grad_b_spline(nu + 1 / 2 - k, degree)
                for k in range(
                    math.ceil(nu - degree / 2),
                    math.floor(nu + degree / 2) + 2
                )
            )
            for nu in abscissa
        ],
        dtype = float
    )

    # Plot
    plt.plot(abscissa, cos_data - spline_data)
    fig.tight_layout()
    plt.show()

# Interact with the degree
interactive(b_spline_as_cos_plot, degree = (1, 10))

interactive(children=(IntSlider(value=3, description='degree', max=10, min=1), Output()), _dom_classes=('widge…